In [2]:
import os
import time
import inspect
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import multiprocessing as mp
from utils import check_path_exists

In [3]:
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
data_path = os.path.join(current_dir, 'MNIST_data')

In [4]:
from tensorflow.keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

/Users/yurei/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/yurei/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/yurei/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/yurei/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: 

In [27]:
task_list = ['train', 'val']
file_type_list = ['images', 'class']

for task in task_list:
    for file_type in file_type_list:
        check_path_exist(os.path.join(data_path, task, file_type))

complete build /Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/train/images
complete build /Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/train/class
complete build /Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/val/images
complete build /Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/val/class


In [9]:
features_train, features_test, targets_train, targets_test = \
    train_test_split(X_train, Y_train, test_size=0.2, random_state=123)

## 製造成資料夾是答案的 dataset 格式

In [108]:
for task in task_list:
    for i in range(10):
        check_path_exist(os.path.join(data_path, task, f'{i}'))

/Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/train/0 does exist
/Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/train/1 does exist
/Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/train/2 does exist
/Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/train/3 does exist
/Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/train/4 does exist
/Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/train/5 does exist
/Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/train/6 does exist
/Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/train/7 does exist
/Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_notes/CNN/MNIST_data/train/8 does exist
/Users/yurei/Desktop/My_file/study_讀書筆記/computer_vision/pytorch_

In [94]:
def get_image_class_path(data_path, task, _class, filename):
    return os.path.join(data_path, task, f'{int(_class)}', filename)


def save_images(data_list, task, part):
    assert data_list[0].shape[0] == data_list[1].shape[0], 'lengths of data are not equal'

    start_number = int((data_list[0].shape[0] * (part))/2)
    end_number = int((data_list[0].shape[0] * (part+1))/2)

    for i in range(start_number, end_number):
        image_path = get_image_class_path(data_path, task, data_list[1][i], f'MNIST_{i}.jpg')
        cv2.imwrite(image_path, data_list[0][i])

In [118]:
class worker(mp.Process):
    def __init__(self, q, worker_num):
        mp.Process.__init__(self)
        self.queue = q
        self.worker_num = worker_num
    def run(self):
        while not self.queue.empty():
            data = self.queue.get()
            save_images(data[0], data[1], data[2])
            time.sleep(3)

In [126]:
train_data_list = [features_train, targets_train]
val_data_list = [features_test, targets_test]

queue1 = [train_data_list, 'train', 0]
queue2 = [train_data_list, 'train', 1]

queue3 = [val_data_list, 'val', 0]
queue4 = [val_data_list, 'val', 1]

my_queue = mp.Queue()

for queue in [queue1, queue2, queue3, queue4]:
    my_queue.put(queue)

In [130]:
my_worker1 = worker(my_queue, 1)
my_worker2 = worker(my_queue, 2)
my_worker3 = worker(my_queue, 3)
my_worker4 = worker(my_queue, 4)

In [131]:
%%time
my_worker1.start()
my_worker2.start()
my_worker3.start()
my_worker4.start()

my_worker1.join()
my_worker2.join()
my_worker3.join()
my_worker4.join()

CPU times: user 45 ms, sys: 147 ms, total: 192 ms
Wall time: 15.9 s
